In [2]:
import pandas as pd
import re
from utils import mahalanobis_algorithm, valid_params, prepare_params, load_properties

In [4]:
df = pd.read_json("assets/olx_location.json", orient="records", convert_dates=False)
df_apply = df.caracteristicas.apply(pd.Series)
df.caracteristicas.apply(pd.Series).columns

Index(['categoria', 'tipo', 'area_util', 'quartos', 'banheiros',
       'vagas_na_garagem', 'cep', 'municipio', 'bairro', 'logradouro',
       'detalhes_do_imovel', 'detalhes_do_condominio', 'condominio', 'iptu'],
      dtype='object')

In [22]:
df_apply

,categoria,tipo,area_util,quartos,banheiros,vagas_na_garagem,cep,municipio,bairro,logradouro,detalhes_do_imovel,detalhes_do_condominio,condominio,iptu
0,Apartamentos,Venda - apartamento padrão,40m²,None,0,0,49044350,Aracaju,Santa Maria,Avenida A200 (Lot P do Sul),NaN,NaN,NaN,NaN
1,Apartamentos,Venda - apartamento padrão,NaN,3,2,1,49025100,Aracaju,Jardins,Avenida Deputado Sílvio Teixeira,NaN,NaN,NaN,NaN
2,Apartamentos,Venda - apartamento padrão,NaN,3,2,1,49025100,Aracaju,Jardins,NaN,NaN,NaN,NaN,NaN
3,Apartamentos,Venda - apartamento padrão,61m²,2,2,1,49095815,Aracaju,Jabotiana,Rua Nourival da Silva,"Área de serviço, Ar condicionado, Churrasqueir...","Condomínio fechado, Segurança 24h, Portaria, P...",NaN,NaN
4,Apartamentos,Venda - apartamento padrão,NaN,3,2,1,49025100,Aracaju,Jardins,Avenida Deputado Sílvio Teixeira,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074,Apartamentos,Venda - apartamento padrão,78m²,3,1,1,49032490,Aracaju,Farolândia,Avenida Murilo Dantas,"Ar condicionado, Área de serviço, Armários na ...","Condomínio fechado, Elevador, Segurança 24h, P...",R$ 350,R$ 800
5075,Apartamentos,Venda - apartamento padrão,123m²,4,3,1,49048010,Aracaju,Luzia,Avenida Adélia Franco - até 9998 - lado par,"Varanda, Área de serviço, Churrasqueira, Quart...",NaN,R$ 735,NaN
5076,Apartamentos,Venda - apartamento padrão,NaN,2,1,1,49092696,Aracaju,Olaria,Rua Três (Lot Portal Ind de Aju),"Área de serviço, Quarto de serviço","Condomínio fechado, Segurança 24h, Portaria, P...",R$ 175,NaN
5077,Apartamentos,Venda - apartamento padrão,64m²,3,2,1,49040700,Aracaju,Inácio Barbosa,Rua Quirino,"Armários embutidos, Varanda, Área de serviço","Condomínio fechado, Elevador, Segurança 24h, P...",R$ 288,NaN


In [3]:
df = pd.read_json("assets/olx_location.json", orient="records", convert_dates=False)
df = df.drop(["titulo", "descricao", "created_at", "codigo"], axis=1)
df = pd.concat([df.drop(["data_publicacao", "caracteristicas"], axis=1), df['caracteristicas'].apply(pd.Series)], axis=1)
df = df.drop(["tipo", "cep", "municipio", "logradouro", "detalhes_do_imovel", "detalhes_do_condominio", "condominio", "iptu"], axis=1)
df = df.dropna()
df = df.rename({'preco': 'value', 'quartos': 'rooms', 'area_util': 'area'}, axis=1)
# df['bairro'] = df['bairro'].apply(lambda x: float(x == 'Jabotiana'))
df['value'] = df['value'].apply(lambda x: re.sub('[^0-9]','', x))
df['value'] = df['value'].astype(float)
df['rooms'] = df['rooms'].apply(lambda x: re.sub('[^0-9]','', x))
df['rooms'] = df['rooms'].astype(float)
df['area'] = df['area'].apply(lambda x: re.sub('[^0-9]','', x))
df['area'] = df['area'].astype(float)

In [6]:
df.localizacao.head()

3     {'lat': -10.9443913, 'lng': -37.0877802}
6     {'lat': -10.9880742, 'lng': -37.0534508}
11     {'lat': -10.9289225, 'lng': -37.056472}
12    {'lat': -10.8923572, 'lng': -37.0658592}
13     {'lat': -10.942472, 'lng': -37.0631421}
Name: localizacao, dtype: object

In [3]:
params = {'value': 10000.0, 'rooms': 3.0, 'area': 0.5}

weights = {'value': [10.0], 'rooms': [3.0], 'area': [0.5]}

sorted_values = mahalanobis_algorithm(df, params, weights)

In [4]:
sorted_values.iloc[2].name

df.iloc[sorted_values.iloc[2].name]

value                          0
categoria           Apartamentos
area                           0
rooms                          3
banheiros                      0
vagas_na_garagem               0
bairro                   Grageru
Name: 576, dtype: object

In [14]:
df

,value,localizacao,categoria,area,rooms,banheiros,vagas_na_garagem,bairro
3,200001.0,"{'lat': -10.9443913, 'lng': -37.0877802}",Apartamentos,61.0,2.0,2,1,False
6,430000.0,"{'lat': -10.9880742, 'lng': -37.0534508}",Apartamentos,0.0,2.0,2,1,False
11,480000.0,"{'lat': -10.9289225, 'lng': -37.056472}",Apartamentos,140.0,4.0,4,2,False
12,95000.0,"{'lat': -10.8923572, 'lng': -37.0658592}",Apartamentos,45.0,2.0,1,1,False
13,940019.0,"{'lat': -10.942472, 'lng': -37.0631421}",Apartamentos,140.0,3.0,4,2,False
...,...,...,...,...,...,...,...,...
5073,156000.0,"{'lat': -10.9220714, 'lng': -37.0534508}",Apartamentos,0.0,2.0,0,0,False
5074,220000.0,"{'lat': -10.9676621, 'lng': -37.0652424}",Apartamentos,78.0,3.0,1,1,False
5075,280050.0,"{'lat': -10.937662, 'lng': -37.0669489}",Apartamentos,123.0,4.0,3,1,False
5077,0.0,"{'lat': -10.9546401, 'lng': -37.0688142}",Apartamentos,64.0,3.0,2,1,False


In [49]:
df['bairro'].unique()

array(['Jabotiana', 'Atalaia', 'Salgado Filho', 'Lamarão', 'Grageru',
       'Luzia', 'Jardins', 'Farolândia', 'Coroa do Meio', 'Ponto Novo',
       'Suíssa', 'Centro', 'São Conrado', 'Inácio Barbosa', 'São José',
       'Zona de Expansão (Mosqueiro)', 'Treze de Julho',
       'Dezoito do Forte', 'Aeroporto', 'Zona de Expansão (Aruana)',
       'Siqueira Campos', 'Pereira Lobo', 'Jardim Centenário',
       'Santa Maria', 'Dom Luciano', 'Olaria', 'Santos Dumont', 'América',
       'Santo Antônio', 'Getúlio Vargas', 'Cirurgia', 'Industrial',
       "Porto D'Antas", '18 do Forte', 'José Conrado de Araújo',
       'Cidade Nova', 'Zona de Expansão (Robalo)', '13 De Julho',
       'Palestina', 'Soledade', 'Japãozinho'], dtype=object)